# Import des datasets SNCF pour TER & TGV

In [1]:
import pandas as pd

stops_ter = pd.read_csv('../data/input/ter_sncf/stops.csv')
trips_ter = pd.read_csv('../data/input/ter_sncf/trips.csv')
stop_times_ter = pd.read_csv('../data/input/ter_sncf/stop_times.csv')
routes_ter = pd.read_csv('../data/input/ter_sncf/routes.csv')
calendar_dates_ter = pd.read_csv('../data/input/ter_sncf/calendar_dates.csv')

stops_tgv = pd.read_csv('../data/input/tgv_sncf/stops.txt')
trips_tgv = pd.read_csv('../data/input/tgv_sncf/trips.txt')
stop_times_tgv = pd.read_csv('../data/input/tgv_sncf/stop_times.txt')
routes_tgv = pd.read_csv('../data/input/tgv_sncf/routes.txt')
calendar_dates_tgv = pd.read_csv('../data/input/tgv_sncf/calendar_dates.txt')

# Calcul des temps de trajets en fonction des horaires et des trajets enregistrés

In [2]:
stops_filtered_ter = stops_ter[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]
stops_filtered_tgv = stops_tgv[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']]

### Trajets TER

In [3]:
stop_times_trips_ter = pd.merge(stop_times_ter, trips_ter, on='trip_id')

stop_times_trips_ter['departure_time'] = pd.to_timedelta(stop_times_trips_ter['departure_time'])
stop_times_trips_ter['arrival_time'] = pd.to_timedelta(stop_times_trips_ter['arrival_time'])

stop_times_trips_ter['next_stop_id'] = stop_times_trips_ter.groupby('trip_id')['stop_id'].shift(-1)
stop_times_trips_ter['next_departure_time'] = stop_times_trips_ter.groupby('trip_id')['departure_time'].shift(-1)

stop_times_trips_ter['travel_time'] = (stop_times_trips_ter['next_departure_time'] - stop_times_trips_ter['departure_time']).dt.total_seconds() / 60.0

stop_times_trips_ter

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,next_stop_id,next_departure_time,travel_time
0,OCESN105330F3164015:2024-10-11T00:34:04Z,0 days 07:28:00,0 days 07:28:00,StopPoint:OCENavette-87571240,0,NaN,0,1,NaN,FR:Line::85d3579c-996b-4671-89af-839855ede78a:,1,105330,1.0,1,NaN,StopPoint:OCENavette-87571000,0 days 07:33:00,5.0
1,OCESN105330F3164015:2024-10-11T00:34:04Z,0 days 07:33:00,0 days 07:33:00,StopPoint:OCENavette-87571000,1,NaN,1,0,NaN,FR:Line::85d3579c-996b-4671-89af-839855ede78a:,1,105330,1.0,1,NaN,NaN,NaT,NaN
2,OCESN105330F3164119:2024-10-11T00:34:04Z,0 days 07:28:00,0 days 07:28:00,StopPoint:OCENavette-87571240,0,NaN,0,1,NaN,FR:Line::85d3579c-996b-4671-89af-839855ede78a:,2,105330,1.0,2,NaN,StopPoint:OCENavette-87571000,0 days 07:33:00,5.0
3,OCESN105330F3164119:2024-10-11T00:34:04Z,0 days 07:33:00,0 days 07:33:00,StopPoint:OCENavette-87571000,1,NaN,1,0,NaN,FR:Line::85d3579c-996b-4671-89af-839855ede78a:,2,105330,1.0,2,NaN,NaN,NaT,NaN
4,OCESN105342F3519685:2024-10-13T00:47:17Z,0 days 17:23:00,0 days 17:23:00,StopPoint:OCENavette-87571240,0,NaN,0,1,NaN,FR:Line::85d3579c-996b-4671-89af-839855ede78a:,3,105342,1.0,3,NaN,StopPoint:OCENavette-87571000,0 days 17:28:00,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575005,OCESN96767F3502841:2024-10-02T00:44:21Z,0 days 22:33:00,0 days 22:33:00,StopPoint:OCETrain TER-85010082,7,NaN,1,0,NaN,FR:Line::C5996E08-5B80-4EA7-8BD5-268DCBA77858:,18,96767,0.0,62084,NaN,NaN,NaT,NaN
575006,OCESN98129R3420319:2024-10-12T00:35:06Z,0 days 20:50:00,0 days 20:50:00,StopPoint:OCECar TER-87715136,0,NaN,0,1,NaN,FR:Line::984d7034-ec86-434a-a5ea-cd737cc2a692:,3283,98129,0.0,62085,NaN,StopPoint:OCECar TER-87715003,0 days 21:15:00,25.0
575007,OCESN98129R3420319:2024-10-12T00:35:06Z,0 days 21:15:00,0 days 21:15:00,StopPoint:OCECar TER-87715003,1,NaN,0,0,NaN,FR:Line::984d7034-ec86-434a-a5ea-cd737cc2a692:,3283,98129,0.0,62085,NaN,StopPoint:OCECar TER-85042150,0 days 21:51:00,36.0
575008,OCESN98129R3420319:2024-10-12T00:35:06Z,0 days 21:51:00,0 days 21:51:00,StopPoint:OCECar TER-85042150,2,NaN,0,0,NaN,FR:Line::984d7034-ec86-434a-a5ea-cd737cc2a692:,3283,98129,0.0,62085,NaN,StopPoint:OCECar TER-85042218,0 days 22:26:00,35.0


### Trajets TGV

In [4]:
stop_times_trips_tgv = pd.merge(stop_times_tgv, trips_tgv, on='trip_id')

stop_times_trips_tgv['departure_time'] = pd.to_timedelta(stop_times_trips_tgv['departure_time'])
stop_times_trips_tgv['arrival_time'] = pd.to_timedelta(stop_times_trips_tgv['arrival_time'])

stop_times_trips_tgv['next_stop_id'] = stop_times_trips_tgv.groupby('trip_id')['stop_id'].shift(-1)
stop_times_trips_tgv['next_departure_time'] = stop_times_trips_tgv.groupby('trip_id')['departure_time'].shift(-1)

stop_times_trips_tgv['travel_time'] = (stop_times_trips_tgv['next_departure_time'] - stop_times_trips_tgv['departure_time']).dt.total_seconds() / 60.0

stop_times_trips_tgv

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,next_stop_id,next_departure_time,travel_time
0,OCESN12213F2859340:2024-09-25T00:19:05Z,0 days 19:04:00,0 days 19:04:00,StopPoint:OCETGV INOUI-87391003,0,NaN,0,1,NaN,FR:Line::A74B127A-8882-4043-BAAA-F49A6703DE54:,1,12213,1.0,1,NaN,StopPoint:OCETGV INOUI-87581009,0 days 21:14:00,130.0
1,OCESN12213F2859340:2024-09-25T00:19:05Z,0 days 21:14:00,0 days 21:14:00,StopPoint:OCETGV INOUI-87581009,1,NaN,1,0,NaN,FR:Line::A74B127A-8882-4043-BAAA-F49A6703DE54:,1,12213,1.0,1,NaN,NaN,NaT,NaN
2,OCESN12213F3090510:2024-10-08T00:31:46Z,0 days 19:04:00,0 days 19:04:00,StopPoint:OCETGV INOUI-87391003,0,NaN,0,1,NaN,FR:Line::A74B127A-8882-4043-BAAA-F49A6703DE54:,2,12213,1.0,2,NaN,StopPoint:OCETGV INOUI-87581009,0 days 21:20:00,136.0
3,OCESN12213F3090510:2024-10-08T00:31:46Z,0 days 21:14:00,0 days 21:20:00,StopPoint:OCETGV INOUI-87581009,1,NaN,0,1,NaN,FR:Line::A74B127A-8882-4043-BAAA-F49A6703DE54:,2,12213,1.0,2,NaN,StopPoint:OCETGV INOUI-87586008,0 days 22:26:00,66.0
4,OCESN12213F3090510:2024-10-08T00:31:46Z,0 days 22:23:00,0 days 22:26:00,StopPoint:OCETGV INOUI-87586008,2,NaN,0,0,NaN,FR:Line::A74B127A-8882-4043-BAAA-F49A6703DE54:,2,12213,1.0,2,NaN,StopPoint:OCETGV INOUI-87611244,0 days 23:09:00,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26176,OCESN9898F3310613:2024-09-25T00:19:05Z,0 days 11:49:00,0 days 11:52:00,StopPoint:OCETGV INOUI-87182014,7,NaN,0,0,NaN,FR:Line::4FA25873-A63A-4A2D-B62F-EF950E45D8A9:,86,9898,NaN,4823,NaN,StopPoint:OCETGV INOUI-87212027,0 days 12:31:00,39.0
26177,OCESN9898F3310613:2024-09-25T00:19:05Z,0 days 12:24:00,0 days 12:31:00,StopPoint:OCETGV INOUI-87212027,8,NaN,0,0,NaN,FR:Line::4FA25873-A63A-4A2D-B62F-EF950E45D8A9:,86,9898,NaN,4823,NaN,StopPoint:OCETGV INOUI-87192039,0 days 13:29:00,58.0
26178,OCESN9898F3310613:2024-09-25T00:19:05Z,0 days 13:24:00,0 days 13:29:00,StopPoint:OCETGV INOUI-87192039,9,NaN,0,0,NaN,FR:Line::4FA25873-A63A-4A2D-B62F-EF950E45D8A9:,86,9898,NaN,4823,NaN,StopPoint:OCETGV INOUI-87191007,0 days 13:51:00,22.0
26179,OCESN9898F3310613:2024-09-25T00:19:05Z,0 days 13:48:00,0 days 13:51:00,StopPoint:OCETGV INOUI-87191007,10,NaN,0,0,NaN,FR:Line::4FA25873-A63A-4A2D-B62F-EF950E45D8A9:,86,9898,NaN,4823,NaN,StopPoint:OCETGV INOUI-82001000,0 days 14:15:00,24.0


# Formatage des datasets et extraction du nom de la ville & positions GPS 

### Trajets TER

In [5]:
connections_ter = stop_times_trips_ter[['stop_id', 'next_stop_id', 'travel_time', 'trip_id']].dropna()

connections_ter = pd.merge(connections_ter, stops_filtered_ter, left_on='stop_id', right_on='stop_id')
connections_ter = pd.merge(connections_ter, stops_filtered_ter[['stop_id', 'stop_name']], left_on='next_stop_id', right_on='stop_id', suffixes=('_start', '_end'))

connections_ter = connections_ter.rename(columns={'stop_name_start': 'departure_station', 'stop_name_end': 'arrival_station'})

connections_ter = connections_ter[connections_ter['travel_time'] > 0].drop_duplicates()

connections_ter["trip_type"] = "TER"

connections_ter['departure_city'] = connections_ter["departure_station"].apply(lambda x: x.split(" ")[0] if x.split(" ")[0] not in ["La", "Le", "Les"] else x)
connections_ter['arrival_city'] = connections_ter['arrival_station'].apply(lambda x: x.split(" ")[0] if x.split(" ")[0] not in ["La", "Le"] else x)

connections_ter = connections_ter.merge(stops_filtered_ter, how='left', left_on='stop_id_start', right_on='stop_id', suffixes=('', '_start'))

connections_ter.rename(columns={'stop_lat': 'stop_lat_start', 'stop_lon': 'stop_lon_start'}, inplace=True)

stops_df_ter_end = stops_filtered_ter.rename(columns={'stop_id': 'stop_id_end', 'stop_lat': 'stop_lat_end', 'stop_lon': 'stop_lon_end'})

connections_ter = connections_ter.merge(stops_df_ter_end, how='left', on='stop_id_end', suffixes=('', '_end'))

stops_df_ter_end = stops_filtered_ter.rename(columns={'stop_id_start': 'stop_id_end', 'stop_lat': 'stop_lat_end', 'stop_lon': 'stop_lon_end'})

connections_ter = connections_ter[['stop_id_start', 'stop_id_end', 'travel_time', 'trip_id', 'departure_station',
         'stop_lat_start', 'stop_lon_start', 'stop_lat_end', 'stop_lon_end',
         'arrival_station', 'trip_type', 'departure_city', 'arrival_city']]

connections_ter = connections_ter.loc[:,~connections_ter.columns.duplicated()].copy()

connections_ter

,stop_id_start,stop_id_end,travel_time,trip_id,departure_station,stop_lat_start,stop_lon_start,stop_lat_end,stop_lon_end,arrival_station,trip_type,departure_city,arrival_city
0,StopPoint:OCENavette-87571240,StopPoint:OCENavette-87571000,5.0,OCESN105330F3164015:2024-10-11T00:34:04Z,Saint-Pierre-des-Corps,47.386140,0.723539,47.389875,0.692836,Tours,TER,Saint-Pierre-des-Corps,Tours
1,StopPoint:OCENavette-87571240,StopPoint:OCENavette-87571000,5.0,OCESN105330F3164119:2024-10-11T00:34:04Z,Saint-Pierre-des-Corps,47.386140,0.723539,47.389875,0.692836,Tours,TER,Saint-Pierre-des-Corps,Tours
2,StopPoint:OCENavette-87571240,StopPoint:OCENavette-87571000,5.0,OCESN105342F3519685:2024-10-13T00:47:17Z,Saint-Pierre-des-Corps,47.386140,0.723539,47.389875,0.692836,Tours,TER,Saint-Pierre-des-Corps,Tours
3,StopPoint:OCENavette-87571240,StopPoint:OCENavette-87571000,5.0,OCESN105342F3519686:2024-10-11T00:34:04Z,Saint-Pierre-des-Corps,47.386140,0.723539,47.389875,0.692836,Tours,TER,Saint-Pierre-des-Corps,Tours
4,StopPoint:OCENavette-87571240,StopPoint:OCENavette-87571000,5.0,OCESN105347F3164263:2024-10-11T00:34:04Z,Saint-Pierre-des-Corps,47.386140,0.723539,47.389875,0.692836,Tours,TER,Saint-Pierre-des-Corps,Tours
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512920,StopPoint:OCETrain TER-85010066,StopPoint:OCETrain TER-85010074,3.0,OCESN96767F3502841:2024-10-02T00:44:21Z,Meyrin,46.222350,6.076900,46.220710,6.093892,Vernier,TER,Meyrin,Vernier
512921,StopPoint:OCETrain TER-85010074,StopPoint:OCETrain TER-85010082,5.0,OCESN96767F3502841:2024-10-02T00:44:21Z,Vernier,46.220710,6.093892,46.210220,6.142440,Geneve,TER,Vernier,Geneve
512922,StopPoint:OCECar TER-87715136,StopPoint:OCECar TER-87715003,25.0,OCESN98129R3420319:2024-10-12T00:35:06Z,Frasne,46.857596,6.157767,46.900988,6.353418,Pontarlier,TER,Frasne,Pontarlier
512923,StopPoint:OCECar TER-87715003,StopPoint:OCECar TER-85042150,36.0,OCESN98129R3420319:2024-10-12T00:35:06Z,Pontarlier,46.900988,6.353418,46.942410,6.676570,Travers,TER,Pontarlier,Travers


### Trajets TGV

In [57]:
connections_tgv = stop_times_trips_tgv[['stop_id', 'next_stop_id', 'travel_time', 'trip_id']].dropna()

connections_tgv = pd.merge(connections_tgv, stops_filtered_tgv, left_on='stop_id', right_on='stop_id')
connections_tgv = pd.merge(connections_tgv, stops_filtered_tgv[['stop_id', 'stop_name']], left_on='next_stop_id', right_on='stop_id', suffixes=('_start', '_end'))

connections_tgv = connections_tgv.rename(columns={'stop_name_start': 'departure_station', 'stop_name_end': 'arrival_station'})

connections_tgv = connections_tgv[connections_tgv['travel_time'] > 0].drop_duplicates()

connections_tgv["trip_type"] = "TGV"

connections_tgv['departure_city'] = connections_tgv["departure_station"].apply(lambda x: x.split(" ")[0] if x.split(" ")[0] not in ["La", "Le", "Les"] else x)
connections_tgv['arrival_city'] = connections_tgv['arrival_station'].apply(lambda x: x.split(" ")[0] if x.split(" ")[0] not in ["La", "Le"] else x)

connections_tgv = connections_tgv.merge(stops_filtered_tgv, how='left', left_on='stop_id_start', right_on='stop_id', suffixes=('', '_start'))

connections_tgv.rename(columns={'stop_lat': 'stop_lat_start', 'stop_lon': 'stop_lon_start'}, inplace=True)

stops_df_tgv_end = stops_filtered_tgv.rename(columns={'stop_id': 'stop_id_end', 'stop_lat': 'stop_lat_end', 'stop_lon': 'stop_lon_end'})

connections_tgv = connections_tgv.merge(stops_df_tgv_end, how='left', on='stop_id_end', suffixes=('', '_end'))

stops_df_tgv_end = stops_filtered_tgv.rename(columns={'stop_id_start': 'stop_id_end', 'stop_lat': 'stop_lat_end', 'stop_lon': 'stop_lon_end'})

connections_tgv = connections_tgv[['stop_id_start', 'stop_id_end', 'travel_time', 'trip_id', 'departure_station',
         'stop_lat_start', 'stop_lon_start', 'stop_lat_end', 'stop_lon_end',
         'arrival_station', 'trip_type', 'departure_city', 'arrival_city']]

connections_tgv = connections_tgv.loc[:,~connections_tgv.columns.duplicated()].copy()



,stop_id_start,stop_id_end,travel_time,trip_id,departure_station,stop_lat_start,stop_lon_start,stop_lat_end,stop_lon_end,arrival_station,trip_type,departure_city,arrival_city


# Fusion des deux datasets

In [55]:
connections = pd.concat([connections_tgv, connections_ter], ignore_index=True)

connections = connections.loc[connections.groupby(['departure_station', 'arrival_station'])['travel_time'].idxmin()].reset_index(drop=True)

connections

connections[(connections["departure_city"] == "Bordeaux") & (connections["arrival_city"] == "Marseille")]

,stop_id_start,stop_id_end,travel_time,trip_id,departure_station,stop_lat_start,stop_lon_start,stop_lat_end,stop_lon_end,arrival_station,trip_type,departure_city,arrival_city


# Calcul de la distance en KM entre la ville de départ et la ville d'arrivée (Pas optimisée)

In [ ]:
import geopy.distance

def calculate_distance(row):
    start_coords = (row['stop_lat_start'], row['stop_lon_start'])
    end_coords = (row['stop_lat_end'], row['stop_lon_end'])
    return geopy.distance.geodesic(start_coords, end_coords).kilometers

connections['distance_km'] = connections.apply(calculate_distance, axis=1)

# Génération du Graph d'après le dataset formaté

In [53]:
import networkx as nx

G = nx.DiGraph()

for index, row in connections.iterrows():
    G.add_edge(row['departure_station'], row['arrival_station'], weight=row['travel_time'])

In [54]:
from networkx import shortest_path, shortest_path_length

shortest_path(G, source='Bordeaux Saint-Jean', target='Marseille Saint-Charles', weight="weight"), shortest_path_length(G, source='Bordeaux Saint-Jean', target='Marseille Saint-Charles', weight="weight")

(['Bordeaux Saint-Jean',
  'Toulouse Matabiau',
  'Carcassonne',
  'Lézignan-Corbières',
  'Narbonne',
  'Béziers',
  'Sète',
  'Montpellier Sud de France',
  'Nîmes Pont du Gard',
  'Tarascon-sur-Rhône',
  'Avignon Centre',
  'Avignon TGV',
  'Marseille Saint-Charles'],
 333.0)